<a href="https://colab.research.google.com/github/AditiG10/Classification-of-COVID-19-chest-X-ray-images/blob/master/basic_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D, Flatten
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!git clone https://github.com/vaibhavG321/Classification-of-COVID-19-chest-X-ray-images

Cloning into 'Classification-of-COVID-19-chest-X-ray-images'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 664 (delta 1), reused 2 (delta 0), pack-reused 656
Receiving objects: 100% (664/664), 222.69 MiB | 13.88 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Checking out files: 100% (698/698), done.


In [4]:
cd Classification-of-COVID-19-chest-X-ray-images 

/content/Classification-of-COVID-19-chest-X-ray-images


In [5]:
cd dataset

/content/Classification-of-COVID-19-chest-X-ray-images/dataset


In [6]:
cd training

/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training


In [7]:
!ls

infected  normal


In [8]:
train_normal_path='/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/normal'
train_infected_path='/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected'
test_normal_path='/content/Classification-of-COVID-19-chest-X-ray-images/dataset/testing/normal'
test_infected_path='/content/Classification-of-COVID-19-chest-X-ray-images/dataset/testing/infected'
evaluate_normal_path='/content/Classification-of-COVID-19-chest-X-ray-images/dataset/evaluation/normal'
evaluate_infected_path='/content/Classification-of-COVID-19-chest-X-ray-images/dataset/evaluation/infected'

In [9]:
print('train_normal cases ',len(os.listdir('/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/normal')))
print('train_infected cases ',len(os.listdir('/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected')))
print('test_normal cases ',len(os.listdir('/content/Classification-of-COVID-19-chest-X-ray-images/dataset/testing/normal')))
print('test_infected cases ',len(os.listdir('/content/Classification-of-COVID-19-chest-X-ray-images/dataset/testing/infected')))
print('evaluate_normal cases ',len(os.listdir('/content/Classification-of-COVID-19-chest-X-ray-images/dataset/evaluation/normal')))
print('evaluate_infected cases ',len(os.listdir('/content/Classification-of-COVID-19-chest-X-ray-images/dataset/evaluation/infected')))

train_normal cases  297
train_infected cases  296
test_normal cases  69
test_infected cases  25
evaluate_normal cases  1
evaluate_infected cases  1


In [10]:
IMAGE_SIZE = (224,224)
DIR_NAME1 = '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected'
DIR_NAME2 = '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/normal'
DIR_NAME3 = '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/testing'
DIR_NAME4 = '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/evaluation'

#IMAGE_DIR = DIR_NAME + 'normal/'
imagePaths=[]
for dirname, _, filenames in os.walk(DIR_NAME1):
    for filename in filenames:
      imagePaths.append(os.path.join(dirname, filename))
for dirname, _, filenames in os.walk(DIR_NAME2):
    for filename in filenames:
      imagePaths.append(os.path.join(dirname, filename))
for dirname, _, filenames in os.walk(DIR_NAME3):
    for filename in filenames:
      imagePaths.append(os.path.join(dirname, filename))
for dirname, _, filenames in os.walk(DIR_NAME4):
    for filename in filenames:
      imagePaths.append(os.path.join(dirname, filename))

In [11]:
imagePaths=imagePaths[:-3]
imagePaths

['/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (50).jpeg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (10).jpg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (28).jpg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (37).jpg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train5.jpg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train43.jpeg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (8).jpeg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (25).jpg',
 '/content/Classification-of-COVID-19-chest-X-ray-images/dataset/training/infected/covid_train (103).jpg',
 '/content/Classification-of-COVID-19-chest-X-ray-

In [12]:
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the image, swap color channels, and resize it to be a fixed
	# 224x224 pixels while ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)

In [13]:
data.shape

(686, 224, 224, 3)

In [14]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
# initialize the training data augmentation object
trainAug = ImageDataGenerator(shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,	rotation_range=15,
	fill_mode="nearest")

In [15]:
trainY.argmax(axis=1)

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,

# CNN Model
optimizer=ADAM,
Learning Rate= 3e-3,
Epochs=34,
Batch Size=16.



In [16]:
model = Sequential()
model.add(Conv2D(80, (3, 3),activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3),activation='relu'))


model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

In [17]:
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [18]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 3e-3
EPOCHS = 34
BS = 16
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[early_stopping,model_chkpt])

[INFO] compiling model...
[INFO] training head...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/34
34/34 [==============================] - 77s 2s/step - loss: 0.8631 - accuracy: 0.5432 - val_loss: 0.6915 - val_accuracy: 0.5290
Epoch 2/34
34/34 [==============================] - 78s 2s/step - loss: 0.6918 - accuracy: 0.5282 - val_loss: 0.6915 - val_accuracy: 0.5290
Epoch 3/34
34/34 [==============================] - 77s 2s/step - loss: 0.6916 - accuracy: 0.5376 - val_loss: 0.6915 - val_accuracy: 0.5290
Epoch 4/34
34/34 [==============================] - 77s 2s/step - loss: 0.6923 - accuracy: 0.5357 - val_loss: 0.6915 - val_accuracy: 0.5290
Epoch 5/34
34/34 [==============================] - 78s 2s/step - loss: 0.6917 - accuracy: 0.5282 - val_loss: 0.6915 - val_accuracy: 0.5290
Epoch 6/34
34/34 [==============================] - 78s 2s/step - loss: 0.6915 - accuracy: 0.5320 - val_loss: 0.6915 - val_accuracy: 0.5290
Epoch 7/34
34/34 [================

In [19]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    infected       0.00      0.00      0.00        65
      normal       0.53      1.00      0.69        73

    accuracy                           0.53       138
   macro avg       0.26      0.50      0.35       138
weighted avg       0.28      0.53      0.37       138



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
_, baseline_model_accuracy = model.evaluate(
    testX, testY, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)



Baseline test accuracy: 0.5289855003356934


NameError: ignored

# CNN Model
optimizer=ADAM, Learning Rate= 1e-4, Batch size= 15, Epochs= 35.

In [21]:
model = Sequential()
model.add(Conv2D(80, (3, 3),activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3),activation='relu'))


model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

In [22]:
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [23]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 35
BS = 15
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[early_stopping,model_chkpt])

[INFO] compiling model...
[INFO] training head...
Epoch 1/35
36/36 [==============================] - 77s 2s/step - loss: 0.6250 - accuracy: 0.6529 - val_loss: 0.6182 - val_accuracy: 0.9420
Epoch 2/35
36/36 [==============================] - 78s 2s/step - loss: 0.3477 - accuracy: 0.8612 - val_loss: 0.4338 - val_accuracy: 0.9348
Epoch 3/35
36/36 [==============================] - 78s 2s/step - loss: 0.3097 - accuracy: 0.8668 - val_loss: 0.3539 - val_accuracy: 0.9565
Epoch 4/35
36/36 [==============================] - 79s 2s/step - loss: 0.2815 - accuracy: 0.8874 - val_loss: 0.3259 - val_accuracy: 0.9565
Epoch 5/35
36/36 [==============================] - 77s 2s/step - loss: 0.2731 - accuracy: 0.8893 - val_loss: 0.2487 - val_accuracy: 0.9493
Epoch 6/35
36/36 [==============================] - 77s 2s/step - loss: 0.2209 - accuracy: 0.9156 - val_loss: 0.2225 - val_accuracy: 0.9493
Epoch 7/35
36/36 [==============================] - 78s 2s/step - loss: 0.2469 - accuracy: 0.9250 - val_loss: 

In [24]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    infected       0.98      0.98      0.98        65
      normal       0.99      0.99      0.99        73

    accuracy                           0.99       138
   macro avg       0.99      0.99      0.99       138
weighted avg       0.99      0.99      0.99       138



In [26]:
_, baseline_model_accuracy = model.evaluate(
    testX, testY, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)



Baseline test accuracy: 0.9855072498321533


#CNN Model 
Optimizer= ADAM, Learning Rate= 3e-4, Epochs= 35, Batch Size= 15


In [27]:
model = Sequential()
model.add(Conv2D(80, (3, 3),activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3),activation='relu'))


model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

In [28]:
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [29]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 3e-4
EPOCHS = 35
BS = 15
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[early_stopping,model_chkpt])

[INFO] compiling model...
[INFO] training head...
Epoch 1/35
36/36 [==============================] - 79s 2s/step - loss: 0.6353 - accuracy: 0.6679 - val_loss: 0.5808 - val_accuracy: 0.9348
Epoch 2/35
36/36 [==============================] - 79s 2s/step - loss: 0.4354 - accuracy: 0.7899 - val_loss: 0.3587 - val_accuracy: 0.9493
Epoch 3/35
36/36 [==============================] - 81s 2s/step - loss: 0.3145 - accuracy: 0.8893 - val_loss: 0.3141 - val_accuracy: 0.9638
Epoch 4/35
36/36 [==============================] - 78s 2s/step - loss: 0.2574 - accuracy: 0.8968 - val_loss: 0.1653 - val_accuracy: 0.9493
Epoch 5/35
36/36 [==============================] - 78s 2s/step - loss: 0.2878 - accuracy: 0.8987 - val_loss: 0.2093 - val_accuracy: 0.9783
Epoch 6/35
36/36 [==============================] - 80s 2s/step - loss: 0.2553 - accuracy: 0.9062 - val_loss: 0.1690 - val_accuracy: 0.9420
Epoch 7/35
36/36 [==============================] - 80s 2s/step - loss: 0.3307 - accuracy: 0.8944 - val_loss: 

In [30]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    infected       1.00      0.97      0.98        65
      normal       0.97      1.00      0.99        73

    accuracy                           0.99       138
   macro avg       0.99      0.98      0.99       138
weighted avg       0.99      0.99      0.99       138



In [31]:
_, baseline_model_accuracy = model.evaluate(
    testX, testY, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)



Baseline test accuracy: 0.9855072498321533


# CNN Model 
optimizer= SGD, Learning rate=1e-3, Batch size=15, epochs= 35

In [32]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [33]:
model = Sequential()
model.add(Conv2D(80, (3, 3),activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3),activation='relu'))


model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

In [34]:
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [35]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-3
EPOCHS = 35
BS = 15
# compile our model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[early_stopping,model_chkpt])

[INFO] compiling model...
[INFO] training head...
Epoch 1/35
36/36 [==============================] - 79s 2s/step - loss: 0.6628 - accuracy: 0.6023 - val_loss: 0.6685 - val_accuracy: 0.9203
Epoch 2/35
36/36 [==============================] - 79s 2s/step - loss: 0.6218 - accuracy: 0.7317 - val_loss: 0.6399 - val_accuracy: 0.5725
Epoch 3/35
36/36 [==============================] - 79s 2s/step - loss: 0.5498 - accuracy: 0.7824 - val_loss: 0.6020 - val_accuracy: 0.9493
Epoch 4/35
36/36 [==============================] - 80s 2s/step - loss: 0.5169 - accuracy: 0.7861 - val_loss: 0.5653 - val_accuracy: 0.7609
Epoch 5/35
36/36 [==============================] - 76s 2s/step - loss: 0.4497 - accuracy: 0.8462 - val_loss: 0.4972 - val_accuracy: 0.9493
Epoch 6/35
36/36 [==============================] - 76s 2s/step - loss: 0.3868 - accuracy: 0.8705 - val_loss: 0.4569 - val_accuracy: 0.9275
Epoch 7/35
36/36 [==============================] - 76s 2s/step - loss: 0.3283 - accuracy: 0.8799 - val_loss: 

In [41]:


# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    infected       0.98      0.97      0.98        65
      normal       0.97      0.99      0.98        73

    accuracy                           0.98       138
   macro avg       0.98      0.98      0.98       138
weighted avg       0.98      0.98      0.98       138



In [42]:
_, baseline_model_accuracy = model.evaluate(
    testX, testY, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)



Baseline test accuracy: 0.97826087474823


# CNN Model
Optimizer= SGD, Learning Rate= 3e-4 ,Epochs= 35, Batch size =15

In [43]:
model = Sequential()
model.add(Conv2D(80, (3, 3),activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3),activation='relu'))


model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

In [44]:
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [38]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 3e-4
EPOCHS = 35
BS = 15
# compile our model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[early_stopping,model_chkpt])

[INFO] compiling model...
[INFO] training head...
Epoch 1/35
36/36 [==============================] - 75s 2s/step - loss: 0.7069 - accuracy: 0.4709 - val_loss: 0.6891 - val_accuracy: 0.5290
Epoch 2/35
36/36 [==============================] - 75s 2s/step - loss: 0.6730 - accuracy: 0.6023 - val_loss: 0.6830 - val_accuracy: 0.5725
Epoch 3/35
36/36 [==============================] - 75s 2s/step - loss: 0.6585 - accuracy: 0.6473 - val_loss: 0.6768 - val_accuracy: 0.5290
Epoch 4/35
36/36 [==============================] - 76s 2s/step - loss: 0.6257 - accuracy: 0.7054 - val_loss: 0.6652 - val_accuracy: 0.8696
Epoch 5/35
36/36 [==============================] - 75s 2s/step - loss: 0.6082 - accuracy: 0.7280 - val_loss: 0.6524 - val_accuracy: 0.8913
Epoch 6/35
36/36 [==============================] - 75s 2s/step - loss: 0.5684 - accuracy: 0.7730 - val_loss: 0.6376 - val_accuracy: 0.9275
Epoch 7/35
36/36 [==============================] - 75s 2s/step - loss: 0.5452 - accuracy: 0.7824 - val_loss: 

In [39]:


# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    infected       0.98      0.97      0.98        65
      normal       0.97      0.99      0.98        73

    accuracy                           0.98       138
   macro avg       0.98      0.98      0.98       138
weighted avg       0.98      0.98      0.98       138



In [40]:
_, baseline_model_accuracy = model.evaluate(
    testX, testY, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)



Baseline test accuracy: 0.97826087474823


#CNN Model
optimizer=SGD, Epochs= 35, Batch Size=15, Learning Rate= 2e-4

In [45]:
model = Sequential()
model.add(Conv2D(80, (3, 3),activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3),activation='relu'))


model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))

In [46]:
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
early_stopping = EarlyStopping(monitor='loss', restore_best_weights=False, patience=10)

In [47]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 2e-4
EPOCHS = 35
BS = 15
# compile our model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,callbacks=[early_stopping,model_chkpt])

[INFO] compiling model...
[INFO] training head...
Epoch 1/35
36/36 [==============================] - 81s 2s/step - loss: 0.7015 - accuracy: 0.4634 - val_loss: 0.6939 - val_accuracy: 0.5217
Epoch 2/35
36/36 [==============================] - 80s 2s/step - loss: 0.6952 - accuracy: 0.4934 - val_loss: 0.6913 - val_accuracy: 0.5362
Epoch 3/35
36/36 [==============================] - 79s 2s/step - loss: 0.6820 - accuracy: 0.5629 - val_loss: 0.6896 - val_accuracy: 0.5362
Epoch 4/35
36/36 [==============================] - 81s 2s/step - loss: 0.6769 - accuracy: 0.5835 - val_loss: 0.6879 - val_accuracy: 0.5362
Epoch 5/35
36/36 [==============================] - 79s 2s/step - loss: 0.6670 - accuracy: 0.6210 - val_loss: 0.6864 - val_accuracy: 0.5362
Epoch 6/35
36/36 [==============================] - 80s 2s/step - loss: 0.6663 - accuracy: 0.6116 - val_loss: 0.6848 - val_accuracy: 0.5362
Epoch 7/35
36/36 [==============================] - 80s 2s/step - loss: 0.6664 - accuracy: 0.5929 - val_loss: 

In [48]:


# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    infected       0.97      0.92      0.94        65
      normal       0.93      0.97      0.95        73

    accuracy                           0.95       138
   macro avg       0.95      0.95      0.95       138
weighted avg       0.95      0.95      0.95       138



In [49]:
_, baseline_model_accuracy = model.evaluate(
    testX, testY, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)



Baseline test accuracy: 0.9492753744125366
